# ISAF (إسعاف)  
## Integrate Security Assessments in your dev Flow
```
    EeeiiiiiEEiiiii.....                                             
       \|/                                                           
        n______     .....iiiiiEEiiiieeEE                             
       :~;     :                  \|/                                
-----;``~'  +  ;------------ ______n --------------------------------
     `-@-----@-=            :     :~:                                
=========================== ;  +  '~``; =============================
                            =-@-----@-'                              
jgs------------------------------------------------------------------
                                                                     
                   DEVSECOPS IN A PYTHON NUTSHELL        
```

# Init
## Pre-requisite
#### Get Jupyter token:
```bash
docker logs func_jupyter.1.shw9s15u6co3cuzp5sjft697t 2>&1 | grep token 
10:03:07.492 LabApp] The Jupyter Notebook is running at: 

http://[all ip addresses on your system]:8888/?token=54523693a4c91624e2efebd5e9dde139b784297e30089504 

to login with a token: http://localhost:8888/?token=54523693a4c91624e2efebd5e9dde139b784297e30089504
```

#### Install git, unzip, curl and faas-cli in the Jupyter container:
```bash
docker exec --user root -ti func_jupyter.1.shw9s15u6co3cuzp5sjft697t bash
root@b9300915e6ad:~# apt-get update && apt-get -y install unzip git curl
root@b9300915e6ad:~# curl -sSL https://cli.openfaas.com | sh <- Bad habit /o\
```

#### Change permissions of docker.sock for the sack of this PoC. Bad habit /o\ . Do not try at home production.
```bash
chmod 777 /var/run/docker.sock
```
#### Upload ISAF.iynb to your Jupyter instance, and press play! \o/

## Environment Build

### Testing Application
Screenshots de l'application + Explications

### Deploy Application

In [ ]:
!git clone https://github.com/h-a-t/RedPill

In [ ]:
!pip install docker

In [4]:
import docker
import io
import tarfile
import os

cli = docker.DockerClient(base_url='unix://var/run/docker.sock')
cli.containers.list()

In [ ]:
## Build app image and pull dependencies
os.chdir('/home/jovyan/RedPill/')
cli.images.build(path='./src/php', tag='hat/app')
#zap_img = cli.images.pull('owasp/zap2docker-weekly:latest')
db_img = cli.images.pull('mariadb:latest')
alpine = cli.images.pull('alpine:latest')


In [ ]:
## Create a dedicated network
cli.networks.create("app_net", driver="bridge") ## cant join overlay network. maybe bridging to isolate?
cli.networks.list()

In [ ]:
## Create Volumes
cli.volumes.create(name='db_data', driver='local')
cli.volumes.create(name='db_init', driver='local')
cli.volumes.create(name='app_data', driver='local')

In [ ]:
## Database provisionning
os.chdir('/home/jovyan/RedPill/src/sql')
tarstream = io.BytesIO()
tar = tarfile.TarFile(fileobj=tarstream, mode='w')
tar.add('staging.sql')
tar.close()

tmp=cli.containers.create(
    image='alpine', 
    volumes={'db_init':{'bind': '/data/', 'mode' : 'rw'}})

tarstream.seek(0)
tmp.put_archive(
    path='/data/',
    data=tarstream
)

## Database run
db_cont = cli.containers.run(
    image='mariadb:latest',
    volumes={'db_init':{'bind': '/docker-entrypoint-initdb.d/', 'mode' : 'rw'},
             'db_data':{'bind': '/var/lib/mysql/', 'mode' : 'rw'}
            },
    detach=True,
    name='app_db',
    environment=['MYSQL_RANDOM_ROOT_PASSWORD=yes','MYSQL_USER=user',
                  'MYSQL_PASSWORD=password','MYSQL_DATABASE=sqli']
        )
db_cont?

In [ ]:
## Webserver provisionning

os.chdir('/home/jovyan/RedPill/src/php')
tarstream = io.BytesIO()
tar = tarfile.TarFile(fileobj=tarstream, mode='w')
tar.add('.')
tar.close()

tmp=cli.containers.create(
    image='alpine', 
    volumes={'app_data':{'bind': '/data/', 'mode' : 'rw'}})

tarstream.seek(0)
tmp.put_archive(
    path='/data/',
    data=tarstream
)

## webserver run
app_cont = cli.containers.run(
    image='hat/app',
    volumes={'app_data':{'bind': '/var/www/html', 'mode' : 'rw'}
            },
    detach=True,
    name='app_web',
    environment=['DB_ENV_MYSQL_USER=user','DB_ENV_MYSQL_PASSWORD=password','BUILD_STAGE=Python'],
    ports={'80/tcp':80},
    links=[('app_db','db')]
        )
app_cont?

# Security Assessment in a Synchronous Execution Flow

## Static Code Analysis


### Push code to SonarQube for code analysis

In [ ]:
## Download Sonarqube scanner
!wget https://sonarsource.bintray.com/Distribution/sonar-scanner-cli/sonar-scanner-cli-3.0.3.778-linux.zip
!unzip sonar-scanner-cli-3.0.3.778-linux.zip

In [ ]:
!./sonar-scanner-3.0.3.778-linux/bin/sonar-scanner -Dsonar.host.url=http://sonarqube:9000 -Dsonar.projectKey=Redpill:latest -Dsonar.sources=RedPill/src/php -Dsonar.language=php

### Push to Clair for a container layer scan

TODO : reg de Jess + Pop instance Clair

## Dynamic Runtime Analysis

### Using the Web-GUI of your favorite Pentesting tool from OWASP: ZAP
- Let L33t do the testing by running a GUI instance of ZAP, just browse localhost:8666/?anonym=true&app=ZAP to start :) 
- Warning, image size: 1,52 Go  
- Cf. https://github.com/zaproxy/zaproxy/wiki/WebSwing  

In [ ]:
zap_img = cli.images.pull('owasp/zap2docker-stable:latest')
scan_cont = cli.containers.run(
    image=zap_img,
    name='app_scan',
    detach=True,
    command="sh -c 'zap-webswing.sh'",
    ports={'8080/tcp':8666, '8090/tcp':8777},
    links=[('app_web','app')]
        )
scan_cont?

### Using the flexibility of your favorite cutting-edge technology: OpenFaas

Dockerfile:  

```
FROM alexellis2/faas-alpinefunction:latest
RUN apk update && apk add nmap
ENV fprocess="xargs nmap"
CMD ["fwatchdog"]
```

nmap_stack:   

```yaml
provider:
  name: faas
  gateway: http://localhost:8080

functions:
  nmap:
    lang: Dockerfile
    handler: ./Dockerfile
    image: hat/nmap
```

# Async

### Example avec faas pour nmap
### Example avec aiodocker pour zap baseline